<a href="https://colab.research.google.com/github/Rahulraj31/Files-Deep_Learning-DL_Tools/blob/main/CustomSentimentAnalysis_DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Sentiment Analysis Using DistilBert : Building and Fine tuning 


dataset Link-https://github.com/krishnaik06/Huggingfacetransformer/blob/main/SMSSpamCollection

In [1]:
import pandas as pd
df=pd.read_csv("/content/SMSSpamCollection", sep="\t", names=["label","message"])

df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
df.shape

(5572, 2)

In [5]:
X=list(df["message"])
y=list(df['label'])

In [6]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [8]:
y[:10]

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [11]:
!pip install transformers

     |████████████████████████████████| 2.5MB 29.6MB/s 
     |████████████████████████████████| 3.3MB 41.5MB/s 
     |████████████████████████████████| 901kB 46.9MB/s 


In [12]:
# Loading Tokenizer
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [13]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)
#by default embedding size is 512

In [17]:
# Converting encodings to dataset object i.e tensors (for tensorflow) 
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [18]:
train_dataset 

<TensorSliceDataset shapes: ({input_ids: (238,), attention_mask: (238,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [19]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

### Note - 

Sequence Classification *(like TFDistilBertForSequenceClassification)* is for sentiment analysis not for question ans. application

**TFTrainingArguments** is responsible for training model.

**TFTrainer** is used for predictions 

In [20]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [21]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.02610751220158168}

In [22]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.4483376, -2.88244  ],
       [-3.404015 ,  3.2009897],
       [ 3.571636 , -2.944173 ],
       ...,
       [ 3.4323187, -2.8194435],
       [-3.4413073,  3.2382665],
       [ 3.4871287, -2.8582046]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.026113547597612655})

- **trainer.predict(test_dataset)[0]** returns prediction score for sentiments

- **trainer.predict(test_dataset)[1]** returns predicted labels

- **trainer.predict(test_dataset)[2**] returns evaluation matrix 

In [27]:
trainer.predict(test_dataset)[1].shape

(1115,)

In [28]:
output=trainer.predict(test_dataset)[1]

In [29]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[955,   0],
       [  0, 160]])

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test, output))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       955
           1       1.00      1.00      1.00       160

    accuracy                           1.00      1115
   macro avg       1.00      1.00      1.00      1115
weighted avg       1.00      1.00      1.00      1115



In [32]:
trainer.save_model('senti_model')